##### Copyright 2019 The TensorFlow Authors.

In [32]:
#The demonstration part is taken from TensorFlow official website

#The exercise code is partially based on the original code

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. If running locally make sure TensorFlow version >= 1.11.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


from sklearn.cluster import KMeans
import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = './shakespeare.txt'

### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
        ])
    return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [22]:
sampled_indices

array([59, 27,  7, 26, 12, 28, 56, 28, 26,  8, 16,  4, 41, 59, 22, 29, 59,
       22, 28,  7,  3, 62, 63, 12, 10, 26, 41, 26, 22, 51, 22, 13,  2, 21,
       12, 42,  9, 36,  6, 52, 26, 22, 56, 51, 23, 13,  0, 44, 46, 21, 18,
       38, 50, 52,  4, 43, 26, 43, 14, 45,  6, 38, 47, 28, 10, 37, 42, 63,
       57, 53, 57, 28, 43, 52, 38, 36, 24, 43, 50, 20, 64, 24, 33, 18, 18,
       59, 38, 32, 39, 42, 37, 59, 64, 35,  0, 51,  0, 55, 36, 28])

Decode these to see the text predicted by this untrained model:

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'so.\n\nYORK:\nI shall not sleep in quiet at the Tower.\n\nGLOUCESTER:\nWhy, what should you fear?\n\nYORK:\nM'

Next Char Predictions: 
 'uO-N?PrPN.D&cuJQuJP-$xy?:NcNJmJA!I?d3X,nNJrmKA\nfhIFZln&eNeBg,ZiP:YdysosPenZXLelHzLUFFuZTadYuzW\nm\nqXP'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.17388


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [25]:

model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [27]:
EPOCHS=10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 23s 135ms/step - loss: 2.7202
Epoch 2/10
172/172 [==============================] - 23s 134ms/step - loss: 1.9767
Epoch 3/10
172/172 [==============================] - 23s 132ms/step - loss: 1.7062
Epoch 4/10
172/172 [==============================] - 23s 132ms/step - loss: 1.5539
Epoch 5/10
172/172 [==============================] - 23s 132ms/step - loss: 1.4631
Epoch 6/10
172/172 [==============================] - 23s 131ms/step - loss: 1.4018
Epoch 7/10
172/172 [==============================] - 23s 132ms/step - loss: 1.3554
Epoch 8/10
172/172 [==============================] - 22s 131ms/step - loss: 1.3164
Epoch 9/10
172/172 [==============================] - 23s 131ms/step - loss: 1.2820
Epoch 10/10
172/172 [==============================] - 23s 131ms/step - loss: 1.2498


## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: What is on,
To charithen imposture you, then's some fengered:
His does that he left coming him: he you neven
'This high about me, Hasters in your daughter cannot
Of Ding Richard from me with this: be
His very unto the Lucour occasion that reprieved you
Of eled agones misilied himself but one:
Nother in Clarence should not call thee make--
Nurse:
We cannot take the e gently vouch beed agreed
Tarn's ear by vorsum, infusion!

First Servingman:
When stage is my fortune lame: break from it?

BRUTUS:
Nothing but done and but every nate;
Against me, and well, go to till ye
On this name, or field to it.

Provost:
I think no many of my Gaunish pipirnw hear?

CAPULET:
Here on a fully silkning loyal sorrow,
As father drightly night by the batter's prodity,--

MARCIUS:
Trast
Is nothing news to cheer horse.

PETRUCHIO:
I sendied a babble?

LUCIO:
I was and gain, and in ammost pass, at honest
Myself dangerous change of ghoses in your sou, and do be
The headst o' the timeless cainted king in
s

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, initialize the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.



In [33]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [34]:
optimizer = tf.keras.optimizers.Adam()

In [35]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

## Exercise TD 3
## Quantization

In this exercise we suggest you implementing the quantization

Try to replace the last layer of the network by another layer that copies its behaviour. At the same time try to reduce its amount of parameters by applying quantization (discussed the last lecture) and retrain it.
For more information see the article: https://arxiv.org/pdf/1510.00149.pdf

<img src="files/quantization.png">


Your task is to take the weight matrix of the last layer of model and apply quantization procedure. As a result you should obtain another layer QuantizedDenseLayer that is parametrized by bias taken from the initial model, by vector of the centroids of the clusters defined on the initial matrix values and by a matrix of cluster indices (see the figure above)

* To define a new layer look at this page: https://www.tensorflow.org/guide/keras/custom_layers_and_models
 - initilize the layer with the vector of bias, means and mean_indices
 - apply clustering algorithms on the kernel matrix of the initial Dense layer of model: *model.layers[-1].kernel*
 - standard clustering algorithm is Kmeans: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
 - implement forward and backward function https://www.tensorflow.org/api_docs/python/tf/custom_gradient
* Implement its retraining
* Test the results by trying regenerate the text


In [ ]:
from tensorflow.keras import layers
from sklearn.cluster import KMeans
from tensorflow.python.keras import activations

class QuantizedDenseLayer(layers.Layer):

    def __init__(self, bias, w=None, means=None, mean_indices=None, n_clusters = 1000, cluster_method = None, activation = None, **kwargs):
        super(QuantizedDenseLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        if cluster_method:
            self.clustering = cluster_method
        else:
            self.clustering = KMeans(n_clusters=self.n_clusters, n_jobs = 1)
            
        self.activation = activations.get(activation)
        
        self.b = tf.Variable(initial_value=bias.numpy(), trainable=True, name = 'bias')
        if (means==None) or (mean_indices==None):
            assert w != None
            compute_means(w)
        else:
            self.means = tf.Variable(initial_value=means.numpy(), dtype = 'float32', trainable =True, name = 'means')    
            self.x_ind = tf.Variable(initial_value=mean_indices.numpy(), dtype = tf.dtypes.int32, trainable = False, name = "means_indices")

    
    def compute_means(self, w):
        #The function that having w as an input tries to build clusters on its values
        #After applying clustering algorithm extract cluster information and save it to self.x_ind
        #Store centroids of each cluster in self.means
        #The expected shapes: self.x_ind.shape = self.w.shape; self.means.shape = (self.n_clusters,)
        #your code goes here

        
        

    def call(self, inputs):
        #implement forward and backward functions
        @tf.custom_gradient
        def w_from_means(m):
            #restore kernel matrix from self.means and self.x_ind
            #your code goes here
            #new_w = ...
            
            #implement the gradient function with respect to means
            def grad(dy):
                #your code goes here
                #y=...
                
    
                return tf.convert_to_tensor(y)
            return new_w, grad
        
        new_w = w_from_means(self.means)
        output = tf.matmul(inputs, new_w)
        
        output = output + self.b
        
        if self.activation is not None:
            return self.activation(output)
        return output
        
        


### Let's build new model

Once the Quantized layer is defined let's rebuild the the new model, where last layer is QuantizedDenseLayer

In [ ]:
def build_Q0model(vocab_size, embedding_dim, rnn_units, bias, means, means_indices, batch_size, n_clusters = 1000):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        QuantizedDenseLayer(bias, means=means, mean_indices=means_indices, n_clusters=n_clusters)
        ])
    return model

Build function that assigns the values of tensors of first layers of *model* to the first layers of *Q0model*.
Additionally, the next function should also call *compute_means* of the last layer of *Q0model* to build *means* and *x_ind* based on the kernel matrix of the Dense layer of *model*

In [ ]:
def transform_toQ0model(model, Q0model):
    #your code goes here
    pass
    

Now as build function is ready and transfer function too, let's use them to build our new *Q0model*

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))


In [ ]:
layer = model.layers[-1]

#you can play with this parameter
NCLUSTERS = 1000

Q0model = build_Q0model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  bias = layer.bias,
  means=tf.zeros((NCLUSTERS)),
  means_indices = tf.zeros(layer.kernel.shape),
  batch_size=BATCH_SIZE, n_clusters = NCLUSTERS)

In [ ]:
transform_toQ0model(model, Q0model)

### Now let's retrain our Q0model so that it returns the good results again

Start from choosing the optimizer again

In [ ]:
optimizer = tf.keras.optimizers.Adam()

Implement *train_Q0step* function in the similar way as *train_step*

In [ ]:
@tf.function
def train_Q0step(inp, target):
    #your code goes here
    #compute loss and apply gradients
    return loss




### Training begins

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dirQ = './training_checkpointsQ'
# Name of the checkpoint files
checkpoint_prefixQ = os.path.join(checkpoint_dirQ, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefixQ,
    save_weights_only=True)

In [ ]:
# Training step
EPOCHS = 3

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initally hidden is None
    hidden = Q0model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_Q0step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        Q0model.save_weights(checkpoint_prefixQ.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Q0model.save_weights(checkpoint_prefixQ.format(epoch=epoch))

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dirQ)

In [ ]:
layer = model.layers[-1]

Q0model =build_Q0model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  bias=layer.bias,
  means=tf.zeros((NCLUSTERS)),
  means_indices=tf.zeros(layer.kernel.shape),
  batch_size=1)

Q0model.load_weights(tf.train.latest_checkpoint(checkpoint_dirQ))

Q0model.build(tf.TensorShape([1, None]))

### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

#### Check if your Quantization and retraing gives good results again

In [ ]:
print(generate_text(Q0model, 'ROMEO'))

### Additionally,

* Try to play with number of cluster and clustering algorithms
* Try to run training for more iterations
* Eventually, you can also apply quantization technique on the bias of Dense layer and also on the weights of GRU layer